# **Is the NBA Becoming a More Global Product: An Analysis of NBA Player Distribution and Trends**

**Introduction**

This project explores the globalisation of the NBA by analysing advanced statistics, player distribution, and trends over time. Using advanced metrics, each player is assigned a composite score to quantify their performance for a given season. By filtering for the top 10 NBA players over the past three seasons based on this composite score, this study aims to evaluate whether globalisation is permeating the league’s elite ranks. Furthermore, this project examines player distribution by nationality over the same period to assess the broader impact of globalisation on the league.

**Key Questions:**
1. Who are the top 10 players in the NBA based on performance over the past three seasons?
2. How has the distribution of NBA players by nationality evolved over the same period?

This analysis leverages data from two primary sources:
- **[Basketball-Reference.com](https://www.basketball-reference.com):** A widely respected resource for historical and current NBA data, providing comprehensive player statistics and advanced metrics.
- **[NBA API](https://developer.nba.com/):** An official source of real-time league data, offering up-to-date player metrics and team statistics for robust and accurate analysis.

By examining these questions and trends, this study seeks to determine whether the influx of international players is reshaping the league and assess whether these players are competing at the highest levels of basketball excellence.

## **Import modules**

In [ ]:
import pandas as pd
import numpy as np
import time
import requests
import os 
import sys
from bs4 import BeautifulSoup
from io import StringIO, BytesIO
from unidecode import unidecode
import unicodedata
from nba_api.stats.endpoints import leaguedashplayerstats, commonplayerinfo
from nba_api.stats.library.parameters import SeasonTypeAllStar
import seaborn as sns
import matplotlib.pyplot as plt
import psycopg2
from psycopg2 import OperationalError
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from PIL import Image, ImageDraw

pd.set_option('display.max_columns', None)

## **Read Environment Variables from Config File**

In [ ]:
# Load environment variables manually
env_file = "config.env"
if os.path.exists(env_file):
    with open(env_file) as f:
        for line in f:
            key, value = line.strip().split("=")
            os.environ[key] = value  # Store in environment


## **1. Data Collection**

### Basketball-Reference Data Scraping

In [ ]:
def transliterate_name(name: str) -> str:
    """
    Ensures names are alphabetic and free of special characters or accents.
    """
    if isinstance(name, str):
        name = unicodedata.normalize("NFKD", name)
        name = unidecode(name)
        name = ''.join(c for c in name if c.isalpha() or c.isspace()).strip()
    return name


def ensure_folder_exists(folder_path: str) -> None:
    """
    Ensures the specified folder exists.
    """
    os.makedirs(folder_path, exist_ok=True)


def scrape_save_stats(season: str, folder_path: str) -> str:
    """
    Scrapes advanced stats from Basketball-Reference for a given season and saves HTML files.
    Returns the file path of the saved HTML.
    """
    ensure_folder_exists(folder_path)
    file_name = f"bball_ref_20{season.split('-')[1]}.html"
    file_path = os.path.join(folder_path, file_name)

    if not os.path.exists(file_path):
        print(f"Scraping data for the {season} season...")
        url = f"https://www.basketball-reference.com/leagues/NBA_20{season.split('-')[1]}_advanced.html"
        response = requests.get(url)
        response.encoding = 'utf-8'
        response.raise_for_status()

        with open(file_path, "w", encoding="utf-8") as f:
            f.write(response.text)
        print(f"File saved: {file_path}")
    else:
        print(f"Data already saved for {season} season.")

    return file_path


def parse_and_clean(file_path: str) -> pd.DataFrame:
    """
    Parses an HTML file, extracts and cleans the advanced stats table, and returns a DataFrame.
    """
    with open(file_path, "r", encoding="utf-8") as f:
        page = f.read()

    soup = BeautifulSoup(page, "html5lib")
    table = soup.find("table", {"id": "advanced"})
    if not table:
        raise ValueError(f"No table found in file: {file_path}")

    df = pd.read_html(StringIO(str(table)))[0]
    df = df[df["Player"] != "Player"]  # Remove duplicate headers
    df = df[["Player", "Team", "G", "PER", "TS%", "USG%", "WS/48", "BPM", "VORP"]]
    df.columns = ["player", "team", "games_played", "per", "ts", "usg", "ws_48", "bpm", "vorp"]
    df["player"] = df["player"].str.strip().apply(transliterate_name)

    return df


def collect_scraped_data(seasons: list[str], folder_path: str) -> pd.DataFrame:
    """
    Collects and compiles scraped advanced stats data for the specified seasons.
    """
    all_stats = []
    for season in seasons:
        try:
            file_path = scrape_save_stats(season, folder_path)
            season_data = parse_and_clean(file_path)
            season_data["season"] = season
            all_stats.append(season_data)
        except Exception as e:
            print(f"Error processing data for {season}: {e}")

    if not all_stats:
        raise ValueError("No valid data collected for any season.")

    return pd.concat(all_stats, ignore_index=True)


last_3_seasons = ["2023-24", "2022-23", "2021-22"]
folder_path = "./data/bball_ref"

bball_ref_df = collect_scraped_data(last_3_seasons, folder_path)

print("Data collection and cleaning completed.")


The `bball_ref_df` includes the following columns:  

| **Column**                       | **Description**                                                                 |
|----------------------------------|---------------------------------------------------------------------------------|
| Player                        | Full name of player                                                                      |
| Team                        | 3-letter team label (e.g. LAL)                                                 |
| Season                      | NBA season (e.g. "2023-24")                                                    |
| Games Played                | Number of games played in the season                                           |
| Player Efficiency Rating (PER) | A per-minute rating summarising accomplishments, adjusted for pace; avg = 15   |
| True Shooting Percentage (TS%) | Accounts for field goals, three-point shots, and free throws                   |
| Usage Percentage (USG%)     | Percentage of team plays used, including FG attempts, FT attempts, and turnovers |
| Win Shares per 48 Minutes (WS/48)| Contribution to team wins, normalised to 48 minutes; league avg = 0.100       |
| Box Plus/Minus (BPM)        | Impact on team's performance per 100 possessions compared to a league-average player |
| Value Over Replacement (VORP) | Contribution above replacement-level player, scaled per season                 |

### **NBA API Data Collection**

In [ ]:
def ensure_folder_exists(folder_path: str) -> None:
    """Ensures the specified folder exists."""
    try:
        os.makedirs(folder_path, exist_ok=True)
        print(f"Folder checked/created: {folder_path}")
    except Exception as e:
        print(f"Error ensuring folder exists: {e}")
        raise

def load_nationality_cache(nationality_filepath: str) -> dict:
    """Loads the nationality cache from a CSV file if it exists."""
    if os.path.exists(nationality_filepath):
        try:
            nationality_df = pd.read_csv(nationality_filepath)
            print(f"Loaded player nationalities from cache: {len(nationality_df)} players.")
            return nationality_df.set_index("player_id")["nationality"].to_dict()
        except Exception as e:
            print(f"Error loading nationality cache: {e}")
            return {}
    print("No nationality cache found; starting fresh.")
    return {}

def save_nationality_cache(nationality_filepath: str, new_nationalities: list[dict]) -> None:
    """Saves the updated nationality cache to a CSV file."""
    if new_nationalities:
        try:
            new_nationalities_df = pd.DataFrame(new_nationalities)
            if os.path.exists(nationality_filepath):
                cached_nationalities = pd.read_csv(nationality_filepath)
                new_nationalities_df = pd.concat(
                    [cached_nationalities, new_nationalities_df], ignore_index=True
                ).drop_duplicates()
            new_nationalities_df.to_csv(nationality_filepath, index=False)
            print(f"Updated player nationalities saved to {nationality_filepath}.")
        except Exception as e:
            print(f"Error saving nationality cache: {e}")
            raise

def fetch_season_stats(season: str, season_filepath: str) -> pd.DataFrame:
    """Fetches and saves advanced stats for a given season."""
    if os.path.exists(season_filepath):
        print(f"Loading data for season {season} from cache.")
        return pd.read_csv(season_filepath)

    print(f"Fetching data for season {season}...")
    try:
        stats = leaguedashplayerstats.LeagueDashPlayerStats(
            season=season,
            season_type_all_star=SeasonTypeAllStar.regular,
            measure_type_detailed_defense="Advanced",
            per_mode_detailed="PerGame"
        ).get_data_frames()[0]

        # Keep only relevant columns
        stats = stats[["PLAYER_ID", "PLAYER_NAME", "TEAM_ABBREVIATION", "GP", "MIN", "OFF_RATING", "DEF_RATING"]]
        stats.columns = ["player_id", "player", "team", "games_played", "min", "off_rating", "def_rating"]
        stats["season"] = season

        # Save to CSV
        stats.to_csv(season_filepath, index=False)
        print(f"Season {season} data saved to {season_filepath}.")
        return stats
    except Exception as e:
        print(f"Error fetching data for season {season}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on failure

def fetch_player_nationalities(unique_player_ids: set, player_nationality_cache: dict) -> list[dict]:
    """Fetches nationalities for unique player IDs and updates the cache."""
    new_nationalities = []

    for player_id in unique_player_ids:
        if player_id not in player_nationality_cache:
            try:
                player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id).get_data_frames()[0]
                nationality = player_info.loc[0, "COUNTRY"]
                player_nationality_cache[player_id] = nationality
                new_nationalities.append({"player_id": player_id, "nationality": nationality})
                print(f"Fetched nationality for player ID {player_id}: {nationality}")
                time.sleep(1)  # Avoid hitting rate limits
            except Exception as e:
                print(f"Error fetching nationality for player ID {player_id}: {e}")
                player_nationality_cache[player_id] = "unknown"
                new_nationalities.append({"player_id": player_id, "nationality": "unknown"})

    return new_nationalities

def fetch_advanced_stats_with_nationality(seasons: list[str], folder_path: str) -> pd.DataFrame:
    """
    Fetches advanced stats and player nationalities, saves data to avoid redundant API calls.
    """
    ensure_folder_exists(folder_path)
    nationality_filepath = os.path.join(folder_path, "player_nationality.csv")
    player_nationality_cache = load_nationality_cache(nationality_filepath)

    all_stats = []
    unique_player_ids = set()

    for season in seasons:
        season_filepath = os.path.join(folder_path, f"nba_stats_{season.replace('-', '_')}.csv")
        season_stats = fetch_season_stats(season, season_filepath)
        if not season_stats.empty:
            all_stats.append(season_stats)
            unique_player_ids.update(season_stats["player_id"].unique())
        time.sleep(2)  # Avoid hitting rate limits

    new_nationalities = fetch_player_nationalities(unique_player_ids, player_nationality_cache)
    save_nationality_cache(nationality_filepath, new_nationalities)

    if not all_stats:
        print("No data was fetched for the specified seasons.")
        raise ValueError("No data was fetched for the specified seasons.")

    combined_stats = pd.concat(all_stats, ignore_index=True)
    combined_stats["nationality"] = combined_stats["player_id"].map(player_nationality_cache)

    return combined_stats

# Execution logic for the notebook
last_3_seasons = ["2023-24", "2022-23", "2021-22"]
folder_path = "./data/nba_api"

nba_api_df = fetch_advanced_stats_with_nationality(last_3_seasons, folder_path)
nba_api_df["player"] = nba_api_df["player"].str.strip().apply(transliterate_name)

print("Data fetching complete.")


The `nba_api_df` DataFrame includes the following columns:

| **Column**                       | **Description**                                                                 |
|----------------------------------|---------------------------------------------------------------------------------|
| Player ID                        | Unique ID assigned to player                                                 |
| Player                        | Full name of player                                                                      |
| Team                        | 3-letter team label (e.g. LAL)                                                 |
| Games Played                | Number of games played in the season                                           |
| Min                | Minutes played per game                                           |
| Offensive Rating                | Estimate of the number of points a player produces per 100 possessions                                           |
| Defensive Rating                | Estimate of the number of points a player allows per 100 possessions                                           |
| Season                      | NBA season (e.g. "2023-24")                                                    |
| Nationality                      | Nationality of player                                                    |

### **Summary of Data Collected**

**Sample Data for Basketball-Reference Data**

| Player            | Team | Games Played | PER  | TS    | USG  | WS/48  | BPM  | VORP | Season  |
|------------------|------|--------------|------|------|------|--------|------|------|---------|
| DeMar DeRozan   | CHI  | 79.0         | 19.7 | 0.584 | 25.8 | 0.147  | 1.8  | 2.8  | 2023-24 |
| Domantas Sabonis| SAC  | 82.0         | 23.2 | 0.637 | 22.2 | 0.206  | 6.5  | 6.2  | 2023-24 |
| Coby White      | CHI  | 79.0         | 14.5 | 0.570 | 22.7 | 0.078  | -0.7 | 0.9  | 2023-24 |
| Mikal Bridges   | BRK  | 82.0         | 14.9 | 0.560 | 24.3 | 0.070  | -0.4 | 1.2  | 2023-24 |
| Paolo Banchero  | ORL  | 80.0         | 17.3 | 0.546 | 29.7 | 0.090  | 1.3  | 2.3  | 2023-24 |

**Sample Data for NBA API Data**

| player_id | Player       | Team | Games Played | Min  | Off Rating | Def Rating | Season  | Nationality |
|-----------|-------------|------|--------------|------|------------|------------|---------|-------------|
| 1630639   | AJ Lawson   | DAL  | 42           | 7.4  | 106.6      | 105.3      | 2023-24 | Canada      |
| 1631260   | AJ Green    | MIL  | 56           | 11.0 | 114.0      | 110.5      | 2023-24 | USA         |
| 1631100   | AJ Griffin  | ATL  | 20           | 8.5  | 106.0      | 120.1      | 2023-24 | USA         |
| 203932    | Aaron Gordon| DEN  | 73           | 31.5 | 119.8      | 111.1      | 2023-24 | USA         |
| 1628988   | Aaron Holiday| HOU  | 78           | 16.3 | 110.5      | 107.6      | 2023-24 | USA         |


## **2. Data Cleaning**

The data cleaning process involves the following steps:

- **Handling Missing Values:**  
  Records with missing values are either removed or imputed using weighted averages to ensure data completeness and reliability.

- **Data Alignment:**  
  Aligning Basketball-Reference data with NBA API data is critical for seamless downstream merging. Key differences addressed include:  
  - **Players on Multiple Teams in One Season:**  
    - NBA API data provides aggregated records for players who played for multiple teams in a season.  
    - Basketball-Reference data includes both aggregated records and individual team records for such players.
    - NBA API data records the team a player last played for during the season in the aggregated record. In contrast, Basketball-Reference uses custom labels, such as `"3TM"`, to indicate that a player participated for three different teams in a single season.
  - **Team Labels:**  
    - Team labels differ between the datasets for certain teams (e.g., Brooklyn Nets recorded as `BRK` in Basketball-Reference and `BKN` in the NBA API).  

These adjustments ensure consistency across datasets and facilitate accurate analysis in subsequent steps.

### **Missing Values: Basketball-Reference Data**

In [ ]:
bball_ref_df.info()

The `bball_ref_df` dataframe contains 2,229 rows and 10 columns. It includes missing values in the following columns: `team`, `games_played`, `per`, `ts`, `ws_48`, `bpm`, and `vorp`. The data type of each column appears to be appropriate for analysis.

To handle missing values, I plan to impute them using a weighted average based on the total number of games played. The imputation process will follow these steps:

1. **Single Record Players:**  
   Players with only one record across all seasons (i.e., those who played for a single team in only one season) will be removed from the dataset.

2. **Players on Multiple Teams in a Single Season:**  
   For players who played for multiple teams in one season, missing values will be imputed using a weighted average, calculated based on the number of games played for each team during that season.

3. **Players with Mixed Records:**  
   For players with missing data in single-team seasons who also played for multiple teams in other season(s), the missing values will be imputed using a weighted average across all their seasons. Any imputed values from step 2 (for multi-team seasons) will be incorporated into this calculation when necessary.


In [ ]:
def find_multi_team_players(df: pd.DataFrame) -> pd.DataFrame:
    """
    Finds and returns rows for players who played for multiple teams in one season.
    """
    multi_team_check = df.groupby(['player', 'season'], as_index=False)['team'].nunique().query('team > 1')
    multi_team_players = df[
        df.set_index(['player', 'season']).index.isin(multi_team_check.set_index(['player', 'season']).index)
    ]
    return multi_team_players


def impute_weighted_avg(df: pd.DataFrame, numeric_cols: list, group_by: list) -> pd.DataFrame:
    """
    Imputes missing numeric values using a weighted average based on games played.
    """
    if 'games_played' not in df.columns:
        raise KeyError("'games_played' column is missing from the DataFrame.")
        
    grouped = df.groupby(group_by)

    for col in numeric_cols:
        if col in df.columns and df[col].isnull().any():
            print(f"Imputing column: {col}")
            df.loc[:, col] = grouped.apply(
                lambda group: group[col].fillna(
                    (group['games_played'] * group[col]).sum() / group['games_played'].sum()
                    if group['games_played'].sum() > 0 else group[col].mean()
                ), include_groups=False
            ).reset_index(level=group_by, drop=True)

    return df


def update_with_imputed_records(original_df: pd.DataFrame, imputed_df: pd.DataFrame, numeric_cols: list) -> pd.DataFrame:
    """
    Updates the original DataFrame with imputed data for numeric columns.
    """
    original_df = original_df.merge(
        imputed_df[['player', 'season', 'team'] + numeric_cols],
        on=['player', 'season', 'team'],
        how='left',
        suffixes=('', '_imputed')
    )
    for col in numeric_cols:
        imputed_col = f'{col}_imputed'
        if imputed_col in original_df.columns:
            original_df[col] = original_df[imputed_col].where(original_df[imputed_col].notnull(), original_df[col])
            original_df.drop(columns=[imputed_col], inplace=True)


    return original_df


def drop_and_impute(df: pd.DataFrame) -> pd.DataFrame:
    """
    Drops certain records, imputes missing values for multi-team players,
    imputes across seasons, and adds aggregated records back.
    """
    # Step 1: Remove Single-Season Players and Records with Missing Games Played
    single_record_players = df.groupby(['player'], as_index=False)['season'].nunique().query('season == 1')
    print(f"Total number of players that only played one season: {single_record_players.shape[0]} players")
    clean_df = df[~df['player'].isin(single_record_players['player'])]
    print(f"Total rows after removing single-season players: {clean_df.shape[0]} rows")

    clean_df = clean_df[clean_df['games_played'].notnull()]
    print(f"Total rows after removing records with missing games played: {clean_df.shape[0]} rows")

    # Separate aggregate rows from non-aggregate rows
    agg_records = clean_df[clean_df['team'].str.contains(r'^\dTM$', na=False)]
    non_agg_records = clean_df[~clean_df['team'].str.contains(r'^\dTM$', na=False)]
    print(f"Aggregated records: {agg_records.shape[0]} rows")
    print(f"Non-aggregated records: {non_agg_records.shape[0]} rows")

    # Step 2: Perform Imputations
    numeric_cols = df.select_dtypes(include=['float64', 'int64']).columns.tolist()

    # Impute for Multi-Team Players
    multi_team_players = find_multi_team_players(non_agg_records)
    print("Columns in multi_team_players:", multi_team_players.columns)
    imputed_multi_df = impute_weighted_avg(multi_team_players, numeric_cols, group_by=['player', 'season'])

    # Update Non-Aggregated Records with Imputed Multi-Team Player Data
    non_agg_records = update_with_imputed_records(non_agg_records, imputed_multi_df, numeric_cols)

    # Impute Across Seasons for All Players
    final_imputed_df = impute_weighted_avg(non_agg_records, numeric_cols, group_by=['player'])

    # Update Non-Aggregated Records with Season-Level Imputations
    non_agg_records = update_with_imputed_records(non_agg_records, final_imputed_df, numeric_cols)

    # Step 3: Add Aggregated Records Back
    clean_df = pd.concat([non_agg_records, agg_records], ignore_index=True)
    print("Missing values have been dropped or imputed.")

    # Debug: Check for Remaining Missing Values
    print("Final DataFrame missing values:\n", clean_df.isnull().sum())

    return clean_df


In [ ]:
bball_imputed_df = drop_and_impute(bball_ref_df)

### **Missing Values: NBA API Data**

In [ ]:
nba_api_df.info()

There are 1716 rows and 8 columns. There are no missing values and the datatype of each column is appropriate. Records for players who only played for one season are dropped.

In [ ]:
nba_dropped_df = drop_and_impute(nba_api_df)

### **Data Alignment: Team Labels of Aggregate Records**

One key difference between the two datasets is how they record players who played for multiple teams in a single season. Basketball-Reference provides both aggregated data and individual statistics for each team, while the NBA API dataset only includes an aggregated record.  

Additionally, while both datasets provide aggregated records, Basketball-Reference labels them using a digit followed by "TM" to indicate the number of teams a player played for in that season. In contrast, the NBA API records only the last team the player played for that season.  

To maintain consistency, we will align the Basketball-Reference dataset with the NBA API dataset.  

In [ ]:
def clean_multi_team_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Handles players who played for multiple teams by assigning the team label
    based on the last occurrence in the records for a single season.

    Args:
        df (pd.DataFrame): Input DataFrame with columns including 'player', 'season',
                           'team', and 'games_played'.

    Returns:
        pd.DataFrame: Cleaned DataFrame with multi-team players assigned the label
                      of the team appearing last in the records for the season.
    """
    # Separate multi-team players
    aggregate_records_mask = df['team'].str.contains(r'^\dTM$', na=False)
    aggregate_records = df[aggregate_records_mask].copy()

    # Identify the team for multi-team players based on the last occurrence
    last_team_mapping = (
        df[~aggregate_records_mask]
        .groupby(['player', 'season'], group_keys=False, as_index=False)
        .tail(1)[['player', 'season', 'team']]  # Use tail(1) to get the last row per group
        .rename(columns={'team': 'last_team'})
    )

    # Create mapping
    mapping = last_team_mapping.set_index(['player', 'season'])['last_team'].to_dict()

    # Map and replace team labels
    aggregate_records['team'] = aggregate_records.set_index(['player', 'season']).index.map(mapping)

    # Identify multi-team players with multiple teams in a single season
    multi_team = (
        df[~aggregate_records_mask]
        .groupby(['player', 'season'])['team']
        .nunique()
        .reset_index()
        .query('team > 1')[['player', 'season']]
    )

    # Filter out rows for multi-team players directly using an index-based approach
    df = df[~aggregate_records_mask]
    df = df[~df.set_index(['player', 'season']).index.isin(multi_team.set_index(['player', 'season']).index)
    ]

    # Combine cleaned data
    df = pd.concat([df, aggregate_records], ignore_index=True).sort_values(by=['player', 'season'])

    return df


# Clean the data
bball_ref_cleaned = clean_multi_team_data(bball_imputed_df)


### **Data Alignment: NBA Team Labels**
Basketball-Reference uses different team labels than the NBA API. To merge the two datasets, these labels need to be standardised. Since the NBA API’s team labels are more intuitive, I will match the Basketball-Reference team labels to those used in the NBA API.

In [ ]:
# Find unique team labels
print("Basketball-Reference Team Labels:")
print(f"{np.sort(bball_ref_cleaned['team'].unique())}\n")
print("NBA API Team Labels:")
print(np.sort(nba_dropped_df['team'].unique()))

The teams with differing labels include: Brooklyn, Charlotte, and Phoenix.

In [ ]:
# Mapping for known differences
team_mapping = {'BRK': 'BKN', 'CHO': 'CHA', 'PHO': 'PHX'}

# Align Basketball-Reference labels to NBA API labels
bball_ref_cleaned['team'] = bball_ref_cleaned['team'].map(team_mapping).fillna(bball_ref_cleaned['team'])

# Find differences in team labels
print(f"Shape of basketball-reference df: {bball_ref_cleaned.shape}")
print(f"Shape of NBA API df: {nba_dropped_df.shape}")

### **Data Alignment: Player Name Suffix**
The two data sources use different naming conventions for player suffixes. To ensure consistency, if a player's name includes a suffix in one dataset but not in the other, the suffix will be added to both.

In [ ]:
def add_suffix(no_suffix_df: pd.DataFrame, suffix_df: pd.DataFrame) -> pd.DataFrame:
    """
    Finds player suffixes from one dataframe and adds them to matching names in the other.
    """
    suffix_dict = {}
    for name in suffix_df['player'].unique():
        if len(name.split(' ')) == 3:
            first_name, last_name, suffix = name.split(' ')
            suffix_dict[f"{first_name} {last_name}"] = suffix
    
    # Add suffix to matching players in no_suffix_df
    no_suffix_df['player'] = no_suffix_df['player'].apply(
        lambda name: f"{name} {suffix_dict[name]}" if name in suffix_dict else name
    )

    return no_suffix_df


In [ ]:
bball_ref_cleaned = add_suffix(bball_ref_cleaned, nba_dropped_df)
nba_cleaned = add_suffix(nba_dropped_df, bball_ref_cleaned)

## **3. Image Processing**  

To enhance player visualisation, a CSV file is generated containing player names, nationalities, and image URLs for their official NBA headshots and national flags. The process follows these steps:  

1. Extract player names, nationalities, and the latest NBA headshot URLs from the NBA API data  
2. Retrieve country flag image URLs from [Country Flag URLs](https://www.kaggle.com/datasets/zhongtr0n/country-flag-urls)  
3. Merge the two datasets on nationality to associate each player with their country’s flag image URL  
4. Transform NBA headshots into circular images using the Python Imaging Library (PIL), upload the processed images to a [GitHub repository](https://github.com/jung-hyung-lee/nba-global-trends/tree/main/visualisation/circular_headshots), and update the CSV with the new URLs of the circular images  
5. Save the final merged dataset as a CSV file for downstream Power BI visualisation  

### **Image Processing Pipeline**  

The code below generates **NBA player headshot URLs**, merges **nationality data with flag images**, and processes **player images into circular PNGs**. It takes `nba_cleaned` as input, assigns **headshot and flag URLs**, downloads images, applies **circular cropping**, and saves the final dataset as `player_headshots.csv`, with processed images stored in `visualisation/circular_headshots/`.

In [ ]:
def process_nba_images(
    nba_df: pd.DataFrame, 
    flag_csv_path: str, 
    output_csv_path: str, 
    image_output_dir: str
):
    """
    Processes NBA player images by generating headshot URLs, merging with flag data, 
    and converting images into circular format.
    """
    
    # Ensure the output directory exists
    os.makedirs(image_output_dir, exist_ok=True)

    # Generate player headshot URLs
    image_df = nba_df[['player_id', 'player', 'nationality']].drop_duplicates()
    image_df['player_id'] = image_df['player_id'].astype(int)
    image_df['player_url'] = image_df['player_id'].apply(
        lambda id: f"https://ak-static.cms.nba.com/wp-content/uploads/headshots/nba/latest/260x190/{id}.png"
    )

    # Load flag data and merge with player information
    country_flags = pd.read_csv(flag_csv_path)
    country_flags.rename(columns={"URL": "flag_url"}, inplace=True)
    merge_on_country = pd.merge(image_df, country_flags, left_on="nationality", right_on="Country", how="left")
    merge_on_code = pd.merge(image_df, country_flags, left_on="nationality", right_on="Code", how="left")
    merge_on_country["flag_url"] = merge_on_country["flag_url"].fillna(merge_on_code["flag_url"])

    # Keep relevant columns
    player_headshot_flag_df = merge_on_country[["player", "player_url", "flag_url"]].drop_duplicates()

    # Function to convert images into circular format
    def make_circular_image(image_url: str, save_path: str):
        """
        Downloads an image from a URL, crops it to a square, applies a circular mask, and saves it as a transparent PNG.
        """
        try:
            response = requests.get(image_url)
            if response.status_code != 200:
                print(f"Failed to download: {image_url}")
                return None

            img = Image.open(BytesIO(response.content)).convert("RGBA")

            # Crop to a square
            width, height = img.size
            size = min(width, height)
            img = img.crop(((width - size) // 2, (height - size) // 2, (width + size) // 2, (height + size) // 2))

            # Apply circular mask
            mask = Image.new("L", (size, size), 0)
            draw = ImageDraw.Draw(mask)
            draw.ellipse((0, 0, size, size), fill=255)

            circular_img = Image.new("RGBA", (size, size), (0, 0, 0, 0))
            circular_img.paste(img, (0, 0), mask)

            # Save processed image
            circular_img.save(save_path, format="PNG")
            return save_path
        except Exception as e:
            print(f"Error processing image {image_url}: {e}")
            return None

    # Process player images
    new_image_urls = []
    for index, row in player_headshot_flag_df.iterrows():
        player_name = row['player'].replace(" ", "_").lower()
        image_url = row['player_url']
        save_path = os.path.join(image_output_dir, f"{player_name}.png").replace("\\", "/")
        processed_path = make_circular_image(image_url, save_path)
        new_image_urls.append(processed_path if processed_path else image_url)

    # Add processed image paths to DataFrame and save
    player_headshot_flag_df["circular_player_url"] = new_image_urls
    player_headshot_flag_df.to_csv(output_csv_path, index=False)

    print(f"Processed images saved in: {image_output_dir}")
    print(f"Updated CSV saved as: {output_csv_path}")


### **Updating Image URLs**

After uploading the circular images, the headshot image URLs are replaced with the new circular versions, and the updated CSV file is saved.

In [ ]:
# Define CSV filepath
csv_filepath = "./data/player_headshots.csv"
csv_savepath = "./data/updated_player_headshots.csv"

# Check if player_headshots.csv file exists and updated_player_headshots.csv file does not exist
if os.path.exists(csv_filepath) and not os.path.exists(csv_savepath):
    # Open player headshots CSV file
    player_url_df = pd.read_csv(csv_filepath)

    # Remove period in front of URLs inside circular_player_url column
    player_url_df["circular_player_url"] = player_url_df["circular_player_url"].str.replace(r"\.", "", n=1, regex=True)

    # Replace player URL column with updated circular image URLs
    player_url_df["player_url"] = player_url_df["circular_player_url"].apply(
        lambda url: f"https://raw.githubusercontent.com/jung-hyung-lee/nba-global-trends/refs/heads/main/{url}"
    )

    # Drop circular_player_url
    player_url_df.drop(columns=["circular_player_url"], inplace=True)

    # Save updated file
    player_url_df.to_csv(csv_savepath, index=False)
    print(f"Updated circular image URLs saved to: {csv_savepath}")

## **4. Exploratory Data Analysis (EDA)**  

After merging the Basketball-Reference and NBA API DataFrames, this section explores underlying patterns and correlations within the data. It also examines the distribution of advanced statistics and provides key descriptive statistics to give a broad overview of league averages over the past three seasons.

### **Merging**  

The two DataFrames from Basketball-Reference and the NBA API will be merged based on `player name`, `season`, `team`, and `games played`. The resulting DataFrame will include all these details along with the **eight collected advanced statistics**.

In [ ]:
merged_diff_df = pd.merge(
    bball_ref_cleaned,
    nba_cleaned,
    on=['player', 'season', 'team', 'games_played'],
    how='outer'
)

As shown below, any remaining unmatched records—based on player name, season, team, and games played—stem from inherent inconsistencies between Basketball-Reference and the NBA API in how they record data. These discrepancies include differing team labels and slight variations in the number of games played. Since the number of such records is insignificant, they will be removed using an inner join.

In [ ]:
missing_data_df = merged_diff_df[merged_diff_df.isnull().any(axis=1)]
print(missing_data_df.head())

In [ ]:
# Merge DataFrames from Basketball-Reference and NBA API
clean_df = pd.merge(
    bball_ref_cleaned,
    nba_cleaned,
    on=['player', 'season', 'team', 'games_played'],
    how='inner'
)
clean_df['games_played'] = clean_df['games_played'].astype(int)
clean_df = clean_df.sort_values(by=['player', 'season'])

# Save final DataFrame as csv
if not os.path.exists("./data/clean_df.csv"):
    clean_df.to_csv("./data/clean_df.csv", index=False)

In [ ]:
clean_df.describe()

### **League-Wide Averages**  

- **`Games Played`**: The average player participated in approximately 51 of 82 games  
- **`PER`**: The mean of 13.6 is slightly below the theoretical league-wide average of 15, likely due to low-minute players dragging the unweighted average down  
- **`TS%`**: The average of 56% aligns well with modern NBA scoring efficiency trends  
- **`USG`**: Represents the percentage of team possessions a player uses. The 18.59% average is slightly below the league average of 20%, consistent with the dataset including bench players  
- **`WS_48`**: The 0.086 average is close to the historical league average of 0.1 for a typical NBA player  
- **`BPM`**: The unweighted mean of -1.12 indicates that the dataset contains many low-minute players  
- **`VORP`**: Suggests most players contribute slightly above replacement-level production  
- **`Offensive Rating`**: Represents an average of 109.66 points scored per 100 possessions while on the court  
- **`Defensive Rating`**: Represents an average of 111.02 points allowed per 100 possessions while on the court  

### **Variance**  

Outside of true shooting percentage, all advanced statistics have high standard deviations, indicating that the data is more spread out from the mean rather than clustering tightly around it. In basketball analytics, high variance suggests significant differences in player performance, with large gaps in production between the best and worst players. If standard deviations were low, it would indicate that most players perform similarly, with few outliers—an inaccurate representation of the league.

### **Outliers**  

Extreme values suggest a mix of limited-minute players and exceptional performances.

In [ ]:
clean_df.info()

In [ ]:
categorical_variables = clean_df.select_dtypes(["object"]).columns.tolist()
# For purposes of EDA, player name and team label columns are removed
categorical_variables.remove('player')
categorical_variables.remove('team')
numeric_variables = clean_df.select_dtypes(['float64', 'int64']).columns.tolist()
print(f"Categorical Variables: {categorical_variables}")
print(f"Numeric Variables: {numeric_variables}")

### **Univariate Analysis: Histograms, Bar Charts, and Boxplots**  

To better understand the distribution of each column and identify any outliers that may be present. 

### **Analysis of Histograms of Numeric Variables**  

- **`Games Played`**: Left-skewed with the majority of players playing more than 55 games but a few playing very few games, pulling the mean down  
- **`PER`**: Approximately normal with most PER scores centered around 13 or 14, very slightly positively skewed  
- **`TS%`**: Approximately normal with most players shooting between 55 and 60%  
- **`USG%`**: Right-skewed with a longer head than tail, most players averaging between 15-18% but a few superstars with extremely high usage pulling the mean up  
- **`WS-48`**: Approximately normal with slight positive skewness  
- **`BPM`**: Approximately normal with slight negative skewness  
- **`VORP`**: Right-skewed with the majority of players having 0 VORP but a few with extremely high ratings pulling the mean up  
- **`MIN`**: Approximately normal with a flat peak, indicating a large spread, which suggests the dataset includes all players from bench players to superstars  
- **`Offensive Rating`**: Approximately normal with slight negative skewness  
- **`Defensive Rating`**: Approximately normal with slight negative skewness

In [ ]:
# Histogram of Numeric Variables
num_bins = int(np.sqrt(len(clean_df)))

# Define grid size
rows = 4
cols = 3

# Create grid of subplots
fig, axes = plt.subplots(rows, cols, figsize=(18, 12))

# Flatten 2D axes array for easier iteration
axes = axes.flatten()

for i, col in enumerate(numeric_variables):
    sns.histplot(clean_df[col], bins=num_bins, ax=axes[i], edgecolor='black')
    axes[i].set_title(f'Histogram of {col}', fontsize=12, fontweight='bold')
    axes[i].set_xlabel(col.capitalize(), fontsize=10)
    axes[i].set_ylabel('count', fontsize=10)

# Remove empty subplots if any
for i in range(len(numeric_variables), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### **Analysis of Boxplots of Numeric Variables**  

- **`Games Played`**: A thicker box indicates a large spread around the median, with 50% of players playing approximately 38 to 68 games in a season  
- **`PER`**: A thin box shows that PER scores cluster closely around the median, slightly below 15. A significant number of positive and negative outliers indicate that the dataset includes players of all calibres  
- **`TS%`**: A thin box suggests that true shooting percentages are tightly distributed around the median of roughly 0.57. There are significantly more outliers with extremely low values, suggesting the unweighted mean may be skewed downward  
- **`USG%`**: A relatively thick box shows that 50% of players use between 15% and 21% of team possessions, with a median of approximately 17%. A high number of outliers with extremely high usage reflects the presence of many superstar players in the NBA  
- **`WS-48`**: A noticeably thin box with a median of approximately 0.1 suggests that most players perform close to the median. However, a greater number of negative outliers likely pulls the mean downward  
- **`BPM`**: A thin box with a median slightly below 0. While there are many positive and negative outliers, the negative ones are both more frequent and more extreme  
- **`VORP`**: A relatively thick box with a median of approximately 0.1. The median is closer to the 1st quartile than the 3rd quartile, indicating larger gaps in VORP between average players and key contributors (e.g., starters) compared to average players and minor contributors (e.g., bench players). This is reinforced by the large number of positive outliers (e.g., star players)  
- **`MIN`**: A noticeably thick box with a median of 20 minutes per game, suggesting high variance in playing time. As expected, no outliers are present  
- **`Offensive Rating` & `Defensive Rating`**: Both have extremely thin boxes with medians of approximately 110 points per 100 possessions. A significant number of negative outliers in both metrics suggest that the unweighted mean may be skewed downward

In [ ]:
# Define figure size
rows = 4
cols = 3

# Create grid of subplots
fig, axes = plt.subplots(rows, cols, figsize=(20, 16))

# Flatten 2D axes array for easier iteration
axes = axes.flatten()

for i, col in enumerate(numeric_variables):
    sns.boxplot(y=clean_df[col], ax=axes[i], color='skyblue')
    axes[i].set_title(f"Boxplot of {col}", fontsize=12, fontweight='bold')
    axes[i].set_ylabel(col.capitalize(), fontsize=10)

for i in range(len(numeric_variables), len(axes)):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

### **Analysis of Bar Charts for Categorical Variables**  

Since the dataset includes a large number of nationalities, only the **top 10 most common** are visualised. The distribution highlights a significant dominance of **American players**, who make up approximately **76.7%** of the dataset. In contrast, the second most common nationality, **Canadian players**, account for only **4.2%**, revealing a stark disparity between the two groups.  

One possible explanation for this gap is that many **foreign players may have played only a single season** before moving to other leagues. As a result, their records would have been excluded from the dataset, contributing to the observed imbalance.

In [ ]:
# Obtain 10 most common nationalities
top_n = 10
top_nationalities = clean_df['nationality'].value_counts().nlargest(top_n)

# Calculate percentages
total_count = clean_df['nationality'].count()
top_nationalities_percentage = (top_nationalities / total_count) * 100

# Define figure size
plt.figure(figsize=(10, 6))
ax = sns.barplot(
    x=top_nationalities.index,
    y=top_nationalities.values,
    palette='pastel',
    edgecolor='black',
    hue=top_nationalities.index,
    legend=False
)

# Annotate bars with percentage values
for i, value in enumerate(top_nationalities.values):
    percentage = top_nationalities_percentage.iloc[i]
    ax.text(i, value + 10, f"{percentage:.1f}%", ha='center', fontsize=10)

plt.title("Top 10 Most Common Nationalities in Dataset", fontsize=14, fontweight='bold')
plt.xlabel('Nationality', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)

plt.show()

The distribution of data across seasons is relatively even, with the 2022-23 season having the highest number of records at approximately 510 and the 2021-22 season the lowest at around 440.  

This fluctuation in player count across seasons is expected, as NBA teams do not have a fixed roster size. Each team can have up to 15 standard contract players, plus two additional two-way players, bringing the maximum roster size to 17.  

Since the dataset includes players who participated in at least two seasons, one possible reason for the higher count in 2022-23 is the overlap of players appearing in both 2021-22 and 2022-23, as well as those in 2022-23 and 2023-24, but not necessarily all three seasons. This overlap may have resulted in a higher number of unique player records in the middle season (2022-23) compared to the other two.

In [ ]:
# Ensure seasons are sorted correctly
season_order = ["2021-22", "2022-23", "2023-24"]

plt.figure(figsize=(10, 6))
sns.countplot(data=clean_df, x='season', order=season_order, hue='season', legend=False, palette='pastel', edgecolor='black')

plt.title("Count Plot of Past 3 NBA Seasons", fontsize=14, fontweight='bold')
plt.xlabel("Season", fontsize=12)
plt.ylabel("Count", fontsize=12)
plt.show()

### **Multivariate Analysis: Heatmap of Correlation Coefficients**  

A heatmap is created to visualise relationships between numerical variables in the dataset. This can highlight the strength and direction of relationships, helping to identify multicollinearity and underlying patterns, providing valuable insights for further analysis.

In [ ]:
# Define DataFrame with only numeric columns
numeric_df = clean_df.select_dtypes(['number'])

# Compute correlation matrix
corr_mat = numeric_df.corr()

# Define figure size
plt.figure(figsize=(10, 6))

# Create heatmap
sns.heatmap(
    corr_mat,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    linewidths=0.5,
    cbar=True
)

plt.title("Heatmap of Correlation Coefficients", fontsize=14, fontweight="bold")
plt.show()

### **Analysis of Heatmap**  

#### **Key Observations**  

- **Strong Positive Correlations:**  
  A strong correlation exists between **Box Plus-Minus (BPM), Win Shares per 48 Minutes (WS/48), and Player Efficiency Rating (PER)**, indicating that higher efficiency ratings align with greater contributions to team success.  
  - **BPM & PER**: **0.89**  
  - **WS/48 & BPM**: **0.87**  
  - **WS/48 & PER**: **0.86**  
  These relationships are expected, as **BPM and WS/48 reflect overall impact**, while **PER measures individual per-minute efficiency**, making them naturally aligned in high-performing players.  

- **Moderate Correlations:**  
  - **Minutes Played per Game (MIN) & Total Games Played**: **0.63**  
  - **MIN & Value Over Replacement Player (VORP)**: **0.59**  
  Players who log more minutes per game tend to appear in more games overall and accumulate higher **VORP**, a cumulative impact metric. The correlation between **VORP and minutes played** is expected, as greater playing time naturally results in higher total contributions.  

- **Weak or No Correlation:**  
  - **Defensive Rating (def_rating)** shows little correlation with most advanced metrics.  
  - A slight negative correlation with **BPM (-0.05)** suggests that **defensive impact may not be well captured in conventional impact metrics**, potentially indicating a bias toward offensive contributions.  

### **Key Takeaways & Next Steps**  

✔️ **Most advanced statistics are positively correlated, except for Defensive Rating.**  
✔️ **Offensive performance appears to be weighted more heavily than defense in standard advanced stats.**  
✔️ **To ensure a more balanced player evaluation**, **composite score weights** must be adjusted to enhance the influence of defensive metrics.

## **5. Storing, Filtering, and Retrieving Data**

**Connect** to a PostgreSQL database, **create tables** to store cleaned data, and **execute queries** with filters to retrieve and store data in DataFrames.

### **PostgreSQL-Related Functions**

Define `environment variables` and `functions` necessary to store, filter, and retrieve data from a PostgreSQL database.

In [ ]:
# Read environment variables
DB_NAME: Optional[str] = os.getenv("DB_NAME")
DB_USER: Optional[str] = os.getenv("DB_USER")
DB_PASSWORD: Optional[str] = os.getenv("DB_PASSWORD")
DB_HOST: Optional[str] = os.getenv("DB_HOST")
DB_PORT: Optional[str] = os.getenv("DB_PORT")

def connect_db():
    """Establishes a connection to the PostgreSQL database"""
    try:
        conn = psycopg2.connect(
            dbname=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            host=DB_HOST,
            port=DB_PORT
        )
        print("Connection to the database was successful.")
        return conn
    except OperationalError as e:
        print(f"Error: Failed to connect to the PostgreSQL database.\nDetails: {e}")
        return None

def table_exists(cursor: psycopg2.extensions.cursor, table_name: str) -> bool:
    """Checks if a table exists in the PostgreSQL database"""
    query = """
    SELECT EXISTS(
        SELECT FROM information_schema.tables
        WHERE table_name = %s
    );
    """
    cursor.execute(query, (table_name,))
    return cursor.fetchone()[0]

def check_and_create_table(cursor: psycopg2.extensions.cursor, conn: psycopg2.extensions.connection) -> None:
    """Checks if the required tables exist and creates them if not"""
    tables: dict[str, str] = {
        "all_three_seasons": """
            CREATE TABLE all_three_seasons (
                player VARCHAR(50) NOT NULL,
                team VARCHAR(10) NOT NULL,
                games_played INT NOT NULL,
                per FLOAT NOT NULL,
                ts FLOAT NOT NULL,
                usg FLOAT NOT NULL,
                ws_48 FLOAT NOT NULL,
                bpm FLOAT NOT NULL,
                vorp FLOAT NOT NULL,
                season VARCHAR(10) NOT NULL,
                min FLOAT NOT NULL,
                off_rating FLOAT NOT NULL,
                def_rating FLOAT NOT NULL
            );
        """
    }

    for table, create_query in tables.items():
        if not table_exists(cursor, table):
            cursor.execute(create_query)
            conn.commit()
            print(f"Table `{table}` created.")

def load_data(engine: create_engine) -> None:
    """Loads data from CSV files into the PostgreSQL database"""
    datasets: dict[str, str] = {
        "all_three_seasons": "./data/clean_df.csv"
    }
    try:
        for table, csv_path in datasets.items():
            df: pd.DataFrame = pd.read_csv(csv_path)
            df.to_sql(table, engine, if_exists="replace", index=False)
            print(f"Data loaded into `{table}`")
    except Exception as e:
        print(f"Failed to load data into `{table}`: {e}")

def retrieve_data(engine: create_engine, query: str, var_name: str = "DataFrame") -> Optional[pd.DataFrame]:
    """Executes the given SQL query and returns the result as a DataFrame"""
    try:
        with engine.connect() as conn:
            df: pd.DataFrame = pd.read_sql(query, conn)
            print(f"Successfully retrieved data and stored in '{var_name}'")
            return df
    except Exception as e:
        print(f"Error executing query: {e}")
        return None


### **Main Execution**  

This process utilises PostgreSQL functions to:  
1. **Connect to the `nba_players` database**  
2. **Create necessary tables** if they do not already exist  
3. **Load CSV data** into these tables  
4. **Execute SQL queries** to retrieve relevant data  
5. **Store query results as DataFrames** for further analysis  

The `weighted_query` below calculates the **weighted averages** of advanced statistics based on total minutes played, applying filters to ensure only significant contributors are considered. Players must meet the following criteria:  
- **At least 15 minutes per game** (out of 48 minutes per game)  
- **At least 41 games played** (half of an 82-game season)  
- **Participation in at least 2 of the last 3 seasons**

The `main_query` retrieves **all player-season records** meeting the **same criteria** while also:
  - Counting **eligible seasons played** (`seasons_played`)  
  - Including **all season-wise statistics**  

After execution, the script saves results in DataFrames (`weighted_average_df` & `final_df`) for further analysis.

These weighted averages will be used to compute a **composite score**, providing a more comprehensive measure of player performance over the past three seasons to support player rankings.

In [ ]:
# Main execution
conn = connect_db()

# Define query to obtain weighted averages of all advanced statistics based on total minutes played per season
weighted_query = """
    WITH total_minutes_cte AS(
        SELECT
            *,
            (min * games_played) AS total_minutes
        FROM all_three_seasons
        WHERE                  -- Filter for players that played at least 15 minutes per game and 41 out of 82 games per season
            min >= 15 AND
            games_played >= 41
    )
    SELECT
        player,
        nationality,
        CASE
            WHEN nationality = 'USA' THEN 'USA'
            ELSE 'International'
        END AS player_origin,
        SUM(total_minutes * per) / SUM(total_minutes) AS avg_per,       
        SUM(total_minutes * ts) / SUM(total_minutes) AS avg_ts,       
        SUM(total_minutes * usg) / SUM(total_minutes) AS avg_usg,
        SUM(total_minutes * ws_48) / SUM(total_minutes) AS avg_ws_48,
        SUM(total_minutes * bpm) / SUM(total_minutes) AS avg_bpm,
        SUM(total_minutes * vorp) / SUM(total_minutes) AS avg_vorp,
        SUM(total_minutes * off_rating) / SUM(total_minutes) AS avg_off_rating,
        SUM(total_minutes * def_rating) / SUM(total_minutes) AS avg_def_rating
    FROM total_minutes_cte
    WHERE total_minutes > 0
    GROUP BY player, nationality, player_origin
    HAVING COUNT(DISTINCT season) >= 2
    ORDER BY player
    ;
"""

main_query = """
    WITH player_season_counts AS (
        SELECT 
            player, 
            COUNT(DISTINCT season) AS seasons_played
        FROM all_three_seasons
        WHERE min >= 15 
        AND games_played >= 41
        GROUP BY player
    )
    SELECT 
        a.*
        , CASE 
            WHEN a.nationality = 'USA' THEN 'USA' 
            ELSE 'International' 
        END AS player_origin
        , psc.seasons_played
    FROM all_three_seasons a
    JOIN player_season_counts psc 
        ON a.player = psc.player
    WHERE psc.seasons_played >= 2  
    AND a.min >= 15 
    AND a.games_played >= 41;
"""

if conn is None:
    print("Connection failed. Exiting...")
    sys.exit

try:
    cursor = conn.cursor()

    # Check and create tables if they do not exist
    check_and_create_table(cursor, conn)

    # Close psycopg2 connection
    cursor.close()
    conn.close()

    # SQLAlchemy connection for efficient data operations
    engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

    # Load data into tables
    load_data(engine)

    # Execute query and store results in DataFrame
    weighted_avg_df = retrieve_data(engine, weighted_query, var_name="weighted_average_df")
    final_df = retrieve_data(engine, main_query, var_name="final_df")

except Exception as e:
    print(f"Unexpected error: {e}")

finally:
    print("Script execution completed.")

## **6. Ranking Players**

Compute the `composite score` for each player based on **normalised weighted-averages** of advanced statistics to **rank players** that played at least 2 out of the last 3 NBA seasons.

### **Normalisation**

Before calculating the composite score, the weighted-averages must be normalised in order to **prevent large statistics such as PER and BPM from dominating the score**. In other words, normalising the averages ensures fair contribution from all statistics. 

In [ ]:
# Normalise advanced statistics
scaler = MinMaxScaler()
metrics = ['per', 'ts', 'usg', 'ws_48', 'bpm', 'vorp', 'off_rating', 'def_rating']
# Create copy to normalise advanced statistics
normalised_final_df = final_df.copy()
normalised_final_df[metrics] = scaler.fit_transform(final_df[metrics])

# Normalise weighted-averages of all advanced statistics
scaler = MinMaxScaler()
avg_metrics = ['avg_per', 'avg_ts', 'avg_usg', 'avg_ws_48', 'avg_bpm', 'avg_vorp', 'avg_off_rating', 'avg_def_rating']
# Create copy to normalise weighted advanced statistics
normalised_avg_df = weighted_avg_df.copy()
normalised_avg_df[avg_metrics] = scaler.fit_transform(weighted_avg_df[avg_metrics])

### **Composite Score**

The **composite score** is constructed by first **normalising** the advanced statistic to ensure consistency across different metrics.  

The **correlation heatmap** reveals that **defensive rating** has little to no correlation with the other seven statistics, which primarily reward offensive contributions. This highlights a key imbalance, as **modern NBA metrics tend to favour offence-focused players.**  

To address this, the composite score is **weighted 70% towards offensive impact** (distributed across the seven offence-oriented metrics) and **30% towards defensive rating**. This **70:30 split** ensures that **defensive effectiveness receives meaningful consideration**, while still reflecting the league’s offensive emphasis. This approach accounts for **both ends of the court** without disproportionately favouring one over the other.

| **Advanced Statistic**         | **Weight (%)** | **Rationale** |
|---------------------------|------------|--------------------------------------------------------------------------|
| Defensive Rating      | 30%        | Balances offensive bias by ensuring defensive impact is strongly considered |
| PER                  | 8%         | Measures overall impact but highly correlated with BPM and WS/48, requiring reduced weight |
| BPM                  | 8%         | Accounts for box score impact but overlaps significantly with PER and WS/48 |
| VORP                 | 8%         | Captures overall value over replacement level but correlates with PER and BPM |
| True Shooting % (TS%) | 11.5%      | Reflects scoring efficiency, an independent metric not tied to counting stats |
| Usage % (USG%)       | 11.5%      | Shows offensive role and involvement, providing unique insight |
| Win Shares per 48 (WS/48) | 11.5%  | Rewards consistent contributions but is highly tied to PER and BPM |
| Offensive Rating      | 11.5%      | Measures offensive efficiency directly, adding a distinct dimension |


In [ ]:
# Define weights for each metric
weights = [0.08, 0.115, 0.115, 0.115, 0.08, 0.08, 0.115, 0.3]
main_metric_weight = dict(zip(metrics, weights))
weighted_metric_weight = dict(zip(avg_metrics, weights))

# Calculate composite score for all advanced statistics
final_df["composite_score"] = normalised_final_df[
    list(main_metric_weight.keys())
].mul(main_metric_weight).sum(axis=1)
final_df["composite_score"] = final_df["composite_score"].round(4)

# Calculate composite score for all weighted statistics
weighted_avg_df["composite_score"] = normalised_avg_df[
    list(weighted_metric_weight.keys())
].mul(weighted_metric_weight).sum(axis=1)
weighted_avg_df["composite_score"] = weighted_avg_df["composite_score"].round(4)

**Saving the data as csv files**

In [ ]:
final_df.to_csv("./data/final_df.csv", index=False)
weighted_avg_df.to_csv("./data/weighted_df.csv", index=False)